#### Autorzy
Zuzanna Zelek 136835 <br>
Mateusz Ratajczak 136790


#### Notatki
shape[0] -> wysokosc <br>
shape[1] -> szerokosc <br>

<b>inverse Radon</b>
    <ul>
    <li> przez co przechodza kolejne emitery na jednym obrocie, suma tego
    <li> dodawana jest do elementów tablicy które przeszyły
    <li> suma danego emitera w danym kącie dodawana jest do tego, przez co dany emiter przechodzi
    <li> odjąć suma emiterów w pierwszym kącie
    </ul>
Aplikacja powinna umożliwić wygenerowanie sinogramu i obrazu wyjściowego bez <br>
pokazania kroków pośrednich oraz z pokazywaniem (iteracyjnie), np. za pomocą <br>
suwaka służącego do regulacji postępu obrotu emitera(ów) i detektorów <br>
Aplikacja powinna móc pozwolić konfigurować następujące elementy: <br>
a) Krok ∆ α układu emiter/detektor. <br>
b) Dla jednego układu emiter/detektor liczbę detektorów (n). <br>
c) Rozwartość /rozpiętość układu emiter/detektor (l). <br>

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import radon, rescale, iradon
from bresenham import bresenham
import math as mth
import imageio
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Markdown
from IPython import display
import PIL as pil

import pydicom
from pydicom.data import get_testdata_files
import pydicom.data
import os
from datetime import date

In [2]:
#From documentation
class ProgressBar:
    def __init__(self, total, display_id):
        self.total = total
        self._progress = 0
        self.html_width = '60ex'
        self.text_width = 60
        self._display_id = display_id

    def __repr__(self):
        fraction = self.progress / self.total
        filled = '=' * int(fraction * self.text_width)
        rest = ' ' * (self.text_width - len(filled))
        return '[{}{}] {}/{}'.format(
            filled, rest,
            self.progress, self.total,
        )

    def _repr_html_(self):
        return "<progress style='width:{}' max='{}' value='{}'></progress>".format(
            self.html_width, self.total, self.progress)

    def display(self):
        self._display_id.display(self)

    def update(self):
        self._display_id.update(self)

    @property
    def progress(self):
        return self._progress

    @progress.setter
    def progress(self, value):
        self._progress = value
        self.update()
        
    def setProgress(self, value):
        self._progress = value
        self.update()

    def setTotal(self, total):
        self.total = total
        self.update()

    def __iter__(self):
        self.display()
        self._progress = -1 # First iteration is 0
        return self

    def __next__(self):
        """Returns current value and increments display by one."""
        self.progress += 1
        if self.progress <= self.total:
            return self.progress
        else:
            raise StopIteration()


## Functions
Emiter - wylicza pozycje emitera <br>
Detektor - wylicza pozycje detektora <br>
Suma Bresenhama - sumuje piksele przez ktore przechodzi promien <br>
Our Radon - funkcja obliczajaca transformacje Radona <br>
Our Inverse Radon - funkcja obliczajaca odwrotną transformacje Radona <br>

In [3]:
def emiter(promien, krok, deltaL, num, n, lOne):
    kat = krok * deltaL - (num - int((n-1)/2)) * lOne
    pe = (-promien * mth.cos(mth.radians(kat)), -promien * mth.sin(mth.radians(kat)))
    return pe

def detektor(promien, krok, deltaL, num, n, lOne):
    kat = krok * deltaL + (num - int((n-1)/2)) * lOne
    pd = (promien * mth.cos(mth.radians(kat)), promien * mth.sin(mth.radians(kat)))
    return pd

In [4]:
def suma_bresenhama(lista_punktow, obrazek):
    suma = 0.0
    wysokosc, szerokosc = obrazek.shape
    for punkt in lista_punktow:
        if punkt[1] < -szerokosc/2 or punkt[1] >= szerokosc/2:
            continue
        if punkt[0] < -wysokosc/2 or punkt[0] >= wysokosc/2:
            continue
        suma += obrazek[int(punkt[0]+wysokosc/2), int(punkt[1]+szerokosc/2)]
    return suma

In [5]:
def our_radon(image, n, deltaAlpha, lPojedyncze):
    numKatow = int(180/deltaAlpha)
    gif = []
    resultRadon = np.zeros((n, numKatow), dtype='float64')
    
    wysokosc, szerokosc = image.shape
    r = mth.sqrt((wysokosc/2)**2 + (szerokosc/2)**2) #promien_okregu
    
    UserInterface.radonProgressBar.setTotal(numKatow)
    UserInterface.radonProgressBar.setProgress(0)
    for num_kata in range (numKatow):
        UserInterface.radonProgressBar.__next__()
        #numer kroku obrotu
        wynik = []
        for tmp in range (n):
            #punkty emiterow/detektorow sasiednich/rownoleglych
            pe = emiter(r, num_kata, deltaAlpha, tmp, n, lPojedyncze)
            pd = detektor(r, num_kata, deltaAlpha, tmp, n, lPojedyncze)

            bres = list(bresenham(int(pe[0]),int(pe[1]),int(pd[0]),int(pd[1])))
            wynik.append(suma_bresenhama(bres, image))
        resultRadon[:,num_kata] = wynik
        UserInterface.stepRadon.append(normalizeImage(resultRadon.copy()))
    
    resultRadon = normalizeImage(resultRadon)
    
    return np.asarray(resultRadon)

In [6]:
def our_inverse_radon(image, n, deltaAlpha, lPojedyncze):
    numKatow = int(180/deltaAlpha)
    gif = []
    wysokosc, szerokosc = UserInterface.image.shape
    
    result = np.zeros((wysokosc, szerokosc), dtype='float64')
    r = mth.sqrt((wysokosc/2)**2 + (szerokosc/2)**2) #promien_okregu
    
    UserInterface.iRadonProgressBar.setTotal(numKatow)
    UserInterface.iRadonProgressBar.setProgress(0)
    UserInterface.rmse = []
    
    for num_kata in range (numKatow):
        UserInterface.iRadonProgressBar.__next__()
        
        if UserInterface.filter == True:
            image[:,num_kata] = useFilter(image[:,num_kata])
        
        for tmp in range (n):
            #punkty emiterow sasiednich/rownoleglych
            pe = emiter(r, num_kata, deltaAlpha, tmp, n, lPojedyncze)
            pd = detektor(r, num_kata, deltaAlpha, tmp, n, lPojedyncze)

            bres = list(bresenham(int(pe[0]),int(pe[1]),int(pd[0]),int(pd[1])))

            for (x,y) in bres:
                x = x + int (wysokosc/2)
                y = y + int (szerokosc/2)
                if not (x < 0 or x >= wysokosc or y < 0 or y >= szerokosc):
                    result[x][y] += image[tmp][num_kata]
        UserInterface.rmse.append(
            np.sqrt(np.mean((normalizeImage(UserInterface.image) - normalizeImage(result) )**2)))
        
        stepResult = result.copy()
        
        stepResult = stepResult - sum(image[:,0])
        if num_kata > 1:
            stepResult = stepResult / (num_kata-1) 
        UserInterface.stepIRadon.append(normalizeImage(stepResult))
    
    #suma elementów z pierwszego skanu
    
    result = result - sum(image[:,0])
    result = result / (numKatow-1)
            
    result = normalizeImage(result)
    return np.asarray(result)

In [7]:
def saveToDicom(pic, saveName):
    filename = get_testdata_files("CT_small.dcm")[0] 
    ds = pydicom.dcmread(filename) # buduje DICOM na podstawie innego DICOMA
    
    pic = normalizeImage(pic)
    obraz = np.asarray(pic, dtype=np.uint16)
    
    ds.Rows = obraz.shape[0] 
    ds.Columns = obraz.shape[1]
    
    ds.PixelData = obraz.tostring() # przeslanie obrazu
    # nadanie wartości 
    ds.PatientName = UserInterface.widgetPatientName.value #imie i nazwisko
    ds.PatientSex = UserInterface.widgetPatientSex.value #plec
    ds.PatientBirthDate = str(UserInterface.widgetBirthDate.value) #Data urodzenia pacjenta
    ds.StudyDate = str(UserInterface.widgetStudyDate.value) #Data badania
    ds.StudyDescription = UserInterface.widgetStudyDescription.value #opis badania
    ds.save_as("dicom/"+saveName, write_like_original=False)

In [8]:
def readFromDicom(pass_dicom):
    base = os.path.abspath("dicom")
    
    filename= pydicom.data.data_manager.get_files(base,pass_dicom)[0]

    ds = pydicom.dcmread(filename) # pobranie przykładowego obrazu
    displayDicomData.update(Markdown(
        "Imie i nazwisko: " + str(ds.PatientName) + "<br>" +
        "Plec: " + str(ds.PatientSex) + "<br>" +
        "Data urodzenia: " + str(ds.PatientBirthDate) + "<br>" +
        "Data badania: " + str(ds.StudyDate) + "<br>" +
        "Opis badania: " + str(ds.StudyDescription)))

    return ds.pixel_array


In [9]:
def useFilter(line):
    filtr = []
    for i in range (-int(len(line)/2), int(len(line)/2)):
        filtr.append(1) if i == 0 else filtr.append(0) if i % 2 == 0 else filtr.append((-4/np.pi**2)/i**2)
    return np.convolve(line, filtr, "same")

def normalizeImage(image):
    if np.amin(image) < 0:
        image = image - np.amin(image)
    image = image/np.amax(image) * 255
    return image

# User Interface class

In [10]:
class UserInterface:
    
    def __init__(self, widgetFilename, widgetRescale, widgetNumEmiters, widgetAlpha, widgetL, widgetFilter,
                 widgetPatientName, widgetPatientSex, widgetBirthDate,
                 widgetStudyDate, widgetStudyDescription, widgetDicomFilename,
                 radonProgressBar, iRadonProgressBar):
        
        UserInterface.widgetFilename = widgetFilename
        UserInterface.widgetRescale = widgetRescale
        UserInterface.widgetNumEmiters = widgetNumEmiters
        UserInterface.widgetAlpha = widgetAlpha
        UserInterface.widgetL = widgetL
        UserInterface.widgetFilter = widgetFilter
        UserInterface.widgetPatientName = widgetPatientName
        UserInterface.widgetPatientSex = widgetPatientSex
        UserInterface.widgetBirthDate = widgetBirthDate
        UserInterface.widgetStudyDate = widgetStudyDate
        UserInterface.widgetStudyDescription = widgetStudyDescription
        UserInterface.widgetDicomFilename = widgetDicomFilename
        UserInterface.radonProgressBar = radonProgressBar
        UserInterface.iRadonProgressBar = iRadonProgressBar
        
    
    @staticmethod
    def readImage():
        filename = "zdjecia/"+UserInterface.widgetFilename.value
        scale = UserInterface.widgetRescale.value
        try:
            UserInterface.image = imread(filename, as_gray=True).astype('float64')
            UserInterface.image = rescale(UserInterface.image, scale=scale, mode='reflect', multichannel=False)
            return 1
        except:
            #print("Wrong file name. Try again...")
            return 0

    @staticmethod
    def refreshValues():
        UserInterface.n = widgetNumEmiters.value
        UserInterface.dAlpha = widgetAlpha.value
        UserInterface.l = widgetL.value/widgetNumEmiters.value
        UserInterface.filter = widgetFilter.value

    @staticmethod
    def createRadonTransform():
        UserInterface.stepRadon = []
        UserInterface.radon = our_radon(UserInterface.image,
                                        UserInterface.n,
                                        UserInterface.dAlpha,
                                        UserInterface.l)
    
    @staticmethod
    def createInverseRadonTransform():
        UserInterface.stepIRadon = []
        UserInterface.iRadon = our_inverse_radon(UserInterface.radon,
                                                 UserInterface.n,
                                                 UserInterface.dAlpha,
                                                 UserInterface.l)

## UI view

In [11]:
def showImage(button):
    if UserInterface.readImage():
        image = UserInterface.image
        """ FROM IMAGE ARRAY TO IMAGE TO DISPLAY!!!"""
        image = image / np.amax(image) * 255
        image = image.astype(np.uint8)
        image = pil.Image.fromarray(image)
        displayImage.update(image)
    else:
        displayImage.update("Wrong filename")

def saveParameters(button):
    UserInterface.refreshValues()
    displayParameters.update(Markdown("Saved values: <br> "
                                      "Num of emiters: "+ str(UserInterface.n) + "<br>" +
                                      "Alpha: "+ str(UserInterface.dAlpha) + "<br>" +
                                      "L: " + str(UserInterface.l) + "<br>" +
                                      "Filter: " + str(UserInterface.filter)))
    
def showRadonTransform(button):
    UserInterface.createRadonTransform()
    widgetRadonIteration.max = len(UserInterface.stepRadon)-1
    widgetRadonIteration.value = widgetRadonIteration.max
    radon = UserInterface.radon
    """ FROM IMAGE ARRAY TO IMAGE TO DISPLAY!!!"""
    radon = radon / np.amax(radon) * 255
    radon = radon.astype(np.uint8)
    radon = pil.Image.fromarray(radon)
    displayRadon.update(radon)
    
def showRadonPart(steps):
    try:
        radon = UserInterface.stepRadon[steps.new]
        """ FROM IMAGE ARRAY TO IMAGE TO DISPLAY!!!"""
        radon = radon / np.amax(radon) * 255
        radon = radon.astype(np.uint8)
        radon = pil.Image.fromarray(radon)
        displayRadon.update(radon)
    except:
        return
    
def showIRadonTransform(button):
    UserInterface.createInverseRadonTransform()
    widgetIRadonIteration.max = len(UserInterface.stepIRadon)-1
    widgetIRadonIteration.value = widgetIRadonIteration.max
    iRadon = UserInterface.iRadon
    """ FROM IMAGE ARRAY TO IMAGE TO DISPLAY!!!"""
    iRadon = iRadon / np.amax(iRadon) * 255
    iRadon = iRadon.astype(np.uint8)
    iRadon = pil.Image.fromarray(iRadon)
    displayIRadon.update(iRadon)
    
def showIRadonPart(steps):
    try:
        iRadon = UserInterface.stepIRadon[steps.new]
        """ FROM IMAGE ARRAY TO IMAGE TO DISPLAY!!!"""
        iRadon = iRadon / np.amax(iRadon) * 255
        iRadon = iRadon.astype(np.uint8)
        iRadon = pil.Image.fromarray(iRadon)
        displayIRadon.update(iRadon)
    except:
        return
    
def showDicom(button):
    dicom = readFromDicom(UserInterface.widgetDicomFilename.value)
    
    """ FROM IMAGE ARRAY TO IMAGE TO DISPLAY!!!"""
    dicom = dicom / np.amax(dicom) * 255
    dicom = dicom.astype(np.uint8)
    dicom = pil.Image.fromarray(dicom)
    displayDicom.update(dicom)

def saveDicom(button):
    saveToDicom(UserInterface.iRadon, UserInterface.widgetDicomFilename.value)
    
def createRMSE(button):
    dane = np.array(UserInterface.rmse)
    y = np.arange(1,len(dane)+1,1)
    plt.cla()
    plt.plot(y, dane)
    displayRMSE.update(Markdown(str(UserInterface.rmse[-1])))
    plt.show()

In [12]:
%matplotlib notebook

widgetFilename = widgets.Combobox(
    placeholder='Write own or chose from list',
    description='Location of image: zdjecia/',
    options=["CT_ScoutView.jpg", "CT_ScoutView-large.jpg", "Kolo.jpg",
            "Kropka.jpg", "Kwadraty2.jpg", "Paski2.jpg", "SADDLE_PE.JPG",
            "SADDLE_PE-large.JPG", "Shepp_logan.jpg",],
    value='',
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetRescale = widgets.FloatSlider(
    value=0.04,
    min=0.01,
    max=1,
    step=0.01,
    description='Rescale:',
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetNumEmiters = widgets.IntSlider(
    value=41,
    min=1,
    max=1000,
    step=1,
    description='Number of emiters/detectors:',
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetAlpha = widgets.FloatSlider(
    value=4.39,
    min=0.01,
    max=90,
    step=0.01,
    description='Alpha angle in degrees:',
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetL = widgets.FloatSlider(
    value=90,
    min=0.01,
    max=270,
    step=0.01,
    description='Diffrence in degrees between first and last emiters',
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetFilter = widgets.RadioButtons(
    options =[('Use filter', True), ('Don\'t use filter', False)]
)

widgetRadonIteration = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Set num of steps of Radon:',
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
    #continuous_update = False
)

widgetIRadonIteration = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Set num of steps of inverse Radon:',
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
    #continuous_update = False
)

widgetPatientName = widgets.Text(
    description = "Name",
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetPatientSex = widgets.RadioButtons(
    options =[('Woman', 'W'), ('Man', 'M')]
)

widgetBirthDate = widgets.DatePicker(
    description = "Birth date",
    value = date(1998, 1, 1),
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetStudyDate = widgets.DatePicker(
    description = "Study date",
    value = date.today(),
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetStudyDescription = widgets.Textarea(
    description = "Study description",
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

widgetDicomFilename = widgets.Text(
    placeholder='Write name of DICOM image',
    description='Read/write dicom image: dicom/',
    value='',
    style = {'description_width': '35%', 'placeholder_width': '60%'},
    layout = {'width': '90%'}
)

displayImage = display.display(display_id = True)
displayParameters = display.display(display_id = True)
displayRadon = display.display(display_id = True)
displayRadonProgress = display.display(display_id = True)
displayRadonSlider = display.display(display_id = True)
displayIRadon = display.display(display_id = True)
displayIRadonProgress = display.display(display_id = True)
displayIRadonSlider = display.display(display_id = True)
displayDicom = display.display(display_id = True)
displayDicomData = display.display(display_id = True)
displayRMSE = display.display(display_id = True)

radonProgressBar = ProgressBar(total = 100, display_id = displayRadonProgress)
iRadonProgressBar = ProgressBar(total = 100, display_id = displayIRadonProgress)

UserInterface(
    widgetFilename,
    widgetRescale,
    widgetNumEmiters,
    widgetAlpha,
    widgetL,
    widgetFilter,
    widgetPatientName,
    widgetPatientSex,
    widgetBirthDate,
    widgetStudyDate,
    widgetStudyDescription,
    widgetDicomFilename,
    radonProgressBar,
    iRadonProgressBar
)


buttonImage = widgets.Button(
    description="Read selected image!",
    layout = {'width': '90%'}
)

buttonParameters = widgets.Button(
    description="Save parameters!",
    layout = {'width': '90%'}
)

buttonRadon = widgets.Button(
    description="Create Radon image",
    layout = {'width': '90%'}
)

buttonIRadon = widgets.Button(
    description="Create inverse Radon image",
    layout = {'width': '90%'}
)

buttonDicomRead = widgets.Button(
    description="Read DICOM image",
    layout = {'width': '90%'}
)

buttonDicomWrite = widgets.Button(
    description="Save DICOM image",
    layout = {'width': '90%'}
)

buttonRMSE = widgets.Button(
    description="Show RMSE plot",
    layout = {'width': '90%'}
)

display.display(
    widgetFilename,
    widgetRescale,
    buttonImage)

displayImage.display(Markdown("Load image here"))

display.display(
    widgetNumEmiters,
    widgetAlpha,
    widgetL,
    widgetFilter
)

display.display(buttonParameters)

displayParameters.display(Markdown("Save parameters"))

display.display(buttonRadon)

displayRadonProgress.display(Markdown(""))

displayRadonSlider.display(widgetRadonIteration)

displayRadon.display(Markdown("Load Radon here"))

display.display(buttonIRadon)

displayIRadonProgress.display(Markdown(""))

displayIRadonSlider.display(widgetIRadonIteration)

displayIRadon.display(Markdown("Load Inverse Radon here"))

display.display(Markdown("<center><h1>Dicom parameters</h1></center><br>"))

display.display(widgetPatientName,
                widgetPatientSex,
                widgetBirthDate,
                widgetStudyDate,
                widgetStudyDescription,
                widgetDicomFilename)

display.display(buttonDicomRead)
display.display(buttonDicomWrite)

displayDicom.display(Markdown("Save and show DICOM image"))
displayDicomData.display(Markdown(""))

display.display(buttonRMSE)
displayRMSE.display(Markdown(""))

buttonImage.on_click(showImage)
buttonParameters.on_click(saveParameters)
buttonRadon.on_click(showRadonTransform)
buttonIRadon.on_click(showIRadonTransform)
buttonDicomRead.on_click(showDicom)
buttonDicomWrite.on_click(saveDicom)
buttonRMSE.on_click(createRMSE)

widgetRadonIteration.observe(showRadonPart, names='value')
widgetIRadonIteration.observe(showIRadonPart, names='value')


Combobox(value='', description='Location of image: zdjecia/', layout=Layout(width='90%'), options=('CT_ScoutVi…

FloatSlider(value=0.04, description='Rescale:', layout=Layout(width='90%'), max=1.0, min=0.01, step=0.01, styl…

Button(description='Read selected image!', layout=Layout(width='90%'), style=ButtonStyle())

Load image here

IntSlider(value=41, description='Number of emiters/detectors:', layout=Layout(width='90%'), max=1000, min=1, s…

...

Text(value='', description='Read/write dicom image: dicom/', layout=Layout(width='90%'), placeholder='Write na…

Button(description='Read DICOM image', layout=Layout(width='90%'), style=ButtonStyle())

Button(description='Save DICOM image', layout=Layout(width='90%'), style=ButtonStyle())

Save and show DICOM image

Button(description='Show RMSE plot', layout=Layout(width='90%'), style=ButtonStyle())